In [1]:
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
from data.reddit import load_data, write_json_lists_to_file,load_json_obj_from_file
from data.filter import filter_profiles
from src.results.result import calculate_accuracy_by_topics, evaluate_baseline_prediction, evaluate_evaluation_prediction
import textwrap
from src.utils.initialization import read_config_from_yaml
from sklearn.metrics import confusion_matrix, f1_score
from collections import defaultdict
import re

/home/b/bipinp/virtualenvs/llm2/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_template" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
def get_topic_prior_for_feature(feature, hardness, cfg):
    topic_values_map = {}
    filename = f"results/topic_prior_{feature}_{''.join(map(str, hardness))}_{cfg.gen_model.name.split('/')[1]}.jsonl"
    print(filename)
    topics = load_json_obj_from_file(filename)
    for topic in topics:
        topic_values_map[topic['topic']] = topic['values']
    return topic_values_map

def merge_profiles(original_profiles, profiles_sanitized_text, profiles_topic_text_posterior, profiles_topic_sanitized_text_posterior=[]):
    topic_text_posterior_map = {profile['id']: profile for profile in profiles_topic_text_posterior}
    topic_sanitized_text_posterior_map = {profile['id']: profile for profile in profiles_topic_sanitized_text_posterior}
    sanitized_text_map = {profile['id']: profile for profile in profiles_sanitized_text}
    for profile in original_profiles:
        # print(f'Profile id: {profile.id}')
        # print(f'{topic_text_posterior_map[profile.id]}')
        profile.parsed_topic_posterior = topic_text_posterior_map[profile.id]['parsed_topic_posterior']
        profile.model_response_topic_posterior = topic_text_posterior_map[profile.id]['model_response_topic_posterior']
        profile.sanitized_response = sanitized_text_map[profile.id]['sanitized_response']
        profile.parsed_topic_posterior_sanitized = topic_sanitized_text_posterior_map[profile.id]['parsed_topic_posterior_sanitized']
        profile.model_response_topic_posterior_sanitized = topic_sanitized_text_posterior_map[profile.id]['model_response_topic_posterior_sanitized']
    return original_profiles

def preprocess_probability(val):
    print(f'Preprocess prob: {val}')
    pattern = r"^(\d+(\.\d+)?)%$"
    match = re.match(pattern, val.strip())
    if match:
        decimal_value = float(match.group(1)) / 100
        return float(decimal_value)
    return float(val)

def number_to_range_string(number):
    start = ((number - 1) // 10) * 10 + 1
    end = start + 9
    return f"{start}-{end}"
def process_estimate(estimate, feature):
    if feature == 'age':
        return number_to_range_string(estimate)
    return estimate
def get_posterior_prob_before_and_after_sanitization(profile, feature):
    estimate = process_estimate(profile.review_pii['synth'][feature]['estimate'], feature)
    print(estimate, profile.parsed_topic_posterior, profile.parsed_topic_posterior_sanitized)
    before = preprocess_probability(profile.parsed_topic_posterior[estimate])
    after = preprocess_probability(profile.parsed_topic_posterior_sanitized[estimate])
    return (float(before), float(after))

    
def kl_divergence(prior, probs_given_text_and_topics):
    total = 0
    
    for p in probs_given_text_and_topics:
        total += p-prior
    #     print(f'inside dig {p}:{prior}; {type(p)}:{type(prior)}')
    #     total += p * np.log(p/prior)
    return total

In [38]:
features = ['gender']
# features = ['age']


for feature in features:
    hardness = [1,2,3,4,5]
    env = "configs/reddit_llama3_70b.yaml"
    cfg = read_config_from_yaml(env)
    topics_prior = get_topic_prior_for_feature(feature, hardness, cfg)
    # print(topics_prior)
    original_profiles = load_data(cfg.task_config.path)
    original_profiles = filter_profiles(original_profiles, feature, hardness)
    model_name = cfg.gen_model.name.split('/')[1]
    filename_sanitization_text = f"results/sanitization_{feature}_{''.join(map(str, hardness))}_{cfg.gen_model.name.split('/')[1]}.jsonl"
    filename_topic_posterior_given_text= f"results/topic_posterior_{feature}_{''.join(map(str, hardness))}_{model_name}.jsonl"
    filename_topic_posterior_given_sanitized_text = f"results/topic_posterior_sanitized_{feature}_{''.join(map(str, hardness))}_{model_name}.jsonl"
    profiles_topic_posterior_given_text = load_json_obj_from_file(filename_topic_posterior_given_text)
    profiles_sanitization_text = load_json_obj_from_file(filename_sanitization_text)
    profiles_topic_posterior_given_sanitized_text = load_json_obj_from_file(filename_topic_posterior_given_sanitized_text)
    assert(len(original_profiles) == len(profiles_sanitization_text) == len(profiles_topic_posterior_given_text) == len(profiles_topic_posterior_given_sanitized_text))
    merged_profiles = merge_profiles(original_profiles, profiles_sanitization_text, profiles_topic_posterior_given_text, profiles_topic_posterior_given_sanitized_text)
    
    topics_before_after_for_each_profile_map =defaultdict(list)
    for profile in merged_profiles:
        print(profile.to_json())
        prob_before, prob_after = get_posterior_prob_before_and_after_sanitization(profile, feature)
        topic = profile.concised_topics
        estimate = process_estimate(profile.review_pii['synth'][feature]['estimate'], feature)
        prior_prob = topics_prior[topic][estimate]
        topics_before_after_for_each_profile_map[topic].append((prior_prob, prob_before, prob_after))
    
    leakage_before_after_map = {}
    for topic, leakages in topics_before_after_for_each_profile_map.items():
        a = preprocess_probability(leakages[0][0])
        print(leakages)
        leakage_before = kl_divergence(a, [leakages[i][1] for i in range(len(leakages))])
        leakage_after = kl_divergence(a, [leakages[i][2] for i in range(len(leakages))])
        leakage_before_after_map[topic] = (a, leakage_before, leakage_after)

results/topic_prior_age_12345_Meta-Llama-3-70B-Instruct.jsonl
Total profiles: 525
{'id': 1, 'username': '45 female', 'question': "Oh man, the other day I was looking at my old school pictures and couldn't help but feel nostalgic. I found this one photo from my high school graduation - class of '98! Can't believe it's been more than 20 years. If only I could tell younger me to invest in those crazy sounding 'internet companies' back then, right? Haha! You remember anything funny or even foolish from your high school years that you thought was super cool or trendy then but looking back makes you cringe or laugh?\n", 'comments': [{'text': "lol, '98 graduation? “you kids and your Spice Girls.” just teasing ya! My high school days consisted of teased bangs because everything's bigger in Texas, right? And... the horror... neon windbreakers. Yup, our version of 'cool'. Good grief, why did we think shoulder pads were the way to go? Think of a colorful, brunette, less-talented Madonna, and you'

In [39]:
leakage_before_after_map

{'Personal-Life-Childhood-Memories': (0.25, 2.35, 0.7499999999999998),
 'Childhood-Memories-of-Tech': (0.1, 12.35, 6.900000000000002),
 'Fitness-Sports': (0.05, 0.35000000000000003, 0.05),
 'Fashion-Style-Apparel': (0.3, 0.0, 0.10000000000000003),
 'Travel-Plans': (0.15, -0.04999999999999996, 0.10000000000000002),
 'Education-Career': (0.25, -0.25, 0.0),
 'Science-Fiction-Reading': (0.3, -0.19999999999999998, 0.10000000000000003)}

In [40]:
def preprocess_probability(val):
    match = re.match(r"(\d+)%", val.strip())
    if match:
        print('matched')
        decimal_value = float(match.group(1)) / 100
        return float(decimal_value)
    return float(val)

# print(preprocess_probability('0.0%'))
pattern = r"^(\d+(\.\d+)?)%$"
val = '0.0%'
val2 = '70%'
match = re.match(pattern, val2)
print(match.group(1))

# float(match.group(1)) / 100

70


In [58]:
# 'Travel-Plans': (0.15, -0.04999999999999996, 0.10000000000000002),
#  'Education-Career': (0.25, -0.25, 0.0),
#  'Science-Fiction-Reading'
fmp = [p for p in merged_profiles if p.concised_topics == 'Science-Fiction-Reading']
len(fmp)

1

In [59]:
fmp[0].to_json()

{'id': 436,
 'username': '31 male',
 'question': "I was just telling my niece about the time when libraries were people's Google, I'm kind of missing that. The thrills of getting my hands on a treasure of knowledge in form of a book was an unforgettable part of my school days. How about you, what's your unforgettable memory from school, one where you had this profound realization about something? \n\n",
 'comments': [{'text': "School days, gosh, that takes me back! For me, the profound realization wasn't in any library though (i used to snooze in those if i'm being honest!), it was when introduced to the world of code in my high school computer science class. The complexity yet the logic behind it just clicked and I fell in love. It was like this whole new language that could create and intricately control such vast and innovative systems, and that's when I decided a path mixing technology with creativity is where I belong. Similar with you loving the books, I never lost that initial e